In [3]:
import numpy as np
import os 
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from ipywidgets import interact
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision
from torch import nn
import torchsummary
from torch.utils.data import DataLoader
from collections import defaultdict
from torchvision.utils import make_grid
import json

In [4]:
class PET_dataset():
    def __init__(self,part,neck_dir,body_dir,phase, transformer=None, aug=None, aug_factor=0):
        self.neck_dir=neck_dir
        self.body_dir=body_dir
        self.part=part
        self.phase=phase
        self.transformer=transformer
        self.aug=aug
        self.aug_factor=aug_factor
        if(self.part=="body"):
            self.image_files = sorted([fn for fn in os.listdir(self.body_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.body_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        elif(self.part=="neck"):
            self.image_files = sorted([fn for fn in os.listdir(self.neck_dir+"/"+self.phase+"/image") if fn.endswith("jpg")])
            self.label_files= sorted([lab for lab in os.listdir(self.neck_dir+"/"+self.phase+"/label") if lab.endswith("txt")])
        
        self.auged_img_list, self.auged_label_list=self.make_aug_list(self.image_files, self.label_files)
        
    def __getitem__(self,index):
        if(self.aug==None):
            filename, image = self.get_image(self.part, index)
            bboxes, class_ids = self.get_label(self.part, index)

            if(self.transformer):
                transformed_data=self.transformer(image=image, bboxes=bboxes, class_ids=class_ids)
                image = transformed_data['image']
                bboxes = np.array(transformed_data['bboxes'])
                class_ids = np.array(transformed_data['class_ids'])


            target = {}
    #         print(f'bboxes:{bboxes}\nclass_ids:{class_ids}\nlen_bboxes:{len(bboxes)}\nlen_class_ids:{len(class_ids)}')
    #         print(f'filename: {filename}')
            target["boxes"] = torch.Tensor(bboxes).float()
            target["labels"] = torch.Tensor(class_ids).long()

            ###
            bboxes=torch.Tensor(bboxes).float()
            class_ids=torch.Tensor(class_ids).long()
            target = np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1)
            ###
        else:
            image=self.auged_img_list[index][1]
            target=self.auged_label_list[index]
            filename=self.auged_img_list[index][0]
        return image, target, filename
    
    def __len__(self, ):
        length=0
        if(self.aug==None):
            length=len(self.image_files)
        else:
            length=len(self.auged_img_list)
        return length
    
    def make_aug_list(self,ori_image_list,ori_label_files):
        aug_image_list=[]
        aug_label_list=[]
        
        print(f"start making augmented images-- augmented factor:{self.aug_factor}")
        for i in range(len(ori_image_list)):
            filename, ori_image = self.get_image(self.part, i)
            ori_bboxes, ori_class_ids = self.get_label(self.part, i)
            for j in range(self.aug_factor):
                auged_data=self.aug(image=ori_image, bboxes=ori_bboxes, class_ids=ori_class_ids)
                image = auged_data['image']
                bboxes = np.array(auged_data['bboxes'])
                class_ids = np.array(auged_data['class_ids'])
                
                bboxes=torch.Tensor(bboxes).float()
                class_ids=torch.Tensor(class_ids).long()
                
                aug_image_list.append((filename, image))
                aug_label_list.append(np.concatenate((bboxes, class_ids[:, np.newaxis]), axis=1))
        
        print(f"total length of augmented images: {len(aug_image_list)}")
        
        return aug_image_list, aug_label_list
        
    
    def get_image(self, part, index): # 이미지 불러오는 함수
        filename = self.image_files[index]
        if(part=="body"):
#             print(f"body called!-> {self.part}")
            image_path = self.body_dir+"/"+self.phase+"/image/"+filename
        elif(part=="neck"):
#             print(f"neck called!-> {self.part}")
            image_path = self.neck_dir+"/"+self.phase+"/image/"+filename
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return filename, image
    
    def get_label(self, part, index): # label (box좌표, class_id) 불러오는 함수
        label_filename=self.label_files[index]
        if(part=="body"):
#             print(f"body label called!-> {self.part}")
            label_path = self.body_dir+"/"+self.phase+"/label/"+label_filename
        elif(part=="neck"):
#             print(f"neck label called!-> {self.part}")
            label_path = self.neck_dir+"/"+self.phase+"/label/"+label_filename
        with open(label_path, 'r') as file:
            labels = file.readlines()
        
        class_ids=[]
        bboxes=[]
        for label in labels:
            label=label.replace("\n", "")
            obj=label.split(' ')[0]
            coor=label.split(' ')[1:]
            obj=int(obj)
            coor=list(map(float, coor))
            class_ids.append(obj)
            bboxes.append(coor)
            
        return bboxes, class_ids
    

In [5]:
NECK_PATH = '/home/host_data/PET_data/Neck'
BODY_PATH = '/home/host_data/PET_data/Body'
IMAGE_SIZE = 448
transformer = A.Compose([
            A.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE),
            A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_ids']),
    )
test_dataset=PET_dataset("neck" ,neck_dir=NECK_PATH,body_dir=BODY_PATH,phase='test', transformer=transformer, aug=None)

start making augmented images-- augmented factor:0
total length of augmented images: 0


In [10]:
def XcenYcenWH_to_XminYmin_WH(box):
#     Xmin = box[:, 0] - box[:, 2]/2
#     Ymin = box[:, 1] - box[:, 3]/2
#     W = box[:, 2]
#     H = box[:, 3]
    Xmin = box[0] - box[2]/2
    Ymin = box[1] - box[3]/2
    W = box[2]
    H = box[3]
    return [Xmin,Ymin,W,H]
#     return np.stack((Xmin, Ymin, W, H), axis=1)

In [4]:
for k in range(len(test_dataset)):
    image=test_dataset[k][0]
    target=test_dataset[k][1]
    filename=test_dataset[k][2]
    print(image.shape, target, filename)

torch.Size([3, 448, 448]) [[0.14337599 0.68225801 0.055735   0.145161   0.        ]] shape1_121.jpg
torch.Size([3, 448, 448]) [[0.58001602 0.21947201 0.164094   0.20132001 0.        ]] shape1_148.jpg
torch.Size([3, 448, 448]) [[0.21932    0.32191801 0.16169199 0.171233   0.        ]
 [0.50787699 0.57362998 0.052239   0.14726    0.        ]] shape1_153.jpg
torch.Size([3, 448, 448]) [[0.90198499 0.72297299 0.040529   0.128378   0.        ]] shape1_48.jpg
torch.Size([3, 448, 448]) [[0.50946498 0.227273   0.13662601 0.22558901 0.        ]
 [0.35061699 0.59427601 0.046091   0.124579   0.        ]] shape1_65.jpg
torch.Size([3, 448, 448]) [[0.109323   0.41396099 0.15924101 0.27597401 0.        ]
 [0.83745903 0.63960999 0.042904   0.123377   0.        ]] shape1_80.jpg
torch.Size([3, 448, 448]) [[0.51067299 0.59515601 0.054187   0.16609    0.        ]] shape1_99.jpg
torch.Size([3, 448, 448]) [[0.55207503 0.340996   0.051261   0.19923399 0.        ]] shape2_105.jpg
torch.Size([3, 448, 448]) [[0.

In [12]:
part='neck'
neck_dir = '/home/host_data/PET_data/Neck'
body_dir = '/home/host_data/PET_data/Body'
phase='test'

ImageToId={"shape1_121.jpg": 0,
           "shape1_148.jpg": 1,
           "shape1_153.jpg": 2,
           "shape1_48.jpg": 3,
           "shape1_65.jpg": 4,
           "shape1_80.jpg": 5,
           "shape1_99.jpg": 6,
           "shape2_105.jpg": 7,
           "shape2_127.jpg": 8,
           "shape2_135.jpg": 9,
           "shape2_14.jpg": 10,
           "shape2_151.jpg": 11,
           "shape2_53.jpg": 12,
           "shape2_55.jpg": 13,
           "shape2_70.jpg": 14,
           "shape2_75.jpg": 15,
           "shape2_79.jpg": 16,
           "shape2_81.jpg": 17,
           "shape3_12.jpg": 18,
           "shape3_14.jpg": 19,
           "shape3_15.jpg": 20,
           "shape3_44.jpg": 21,
           "shape4_24.jpg": 22,
           "shape5_34.jpg": 23,
           "shape6_34.jpg": 24,}
with open('/home/host_data/PET_data/Neck/test/test.json', 'r') as f:
        json_data = json.load(f)
        
for k in range(len(test_dataset)):
    image=test_dataset[k][0]
    targets=test_dataset[k][1]
    image_filename=test_dataset[k][2]
    w=image.shape[2]
    h=image.shape[1]
    for target in targets:
        target[0]=target[0]*w
        target[2]=target[2]*w
        target[1]=target[1]*h
        target[3]=target[3]*h
#         class_ids.append(obj)
#         bboxes.append(coor)
    val={"id": 0,
   "image_id": 0,
   "bbox": [],
   "area": 0,
   "iscrowd": 0,
   "category_id": 0,
   "segmentation": []}
    for i in range(len(targets)):
        val={"id": 0,
       "image_id": 0,
       "bbox": [],
       "area": 0,
       "iscrowd": 0,
       "category_id": 0,
       "segmentation": []}
        val['image_id']=ImageToId[image_filename]
        val['id']=i
        val['area']=targets[i][2]*targets[i][3]
        val['category_id']=int(targets[i][4])
#         val['bbox']=targets[i][0:4].tolist()
        val['bbox']=XcenYcenWH_to_XminYmin_WH([targets[i][0],targets[i][1],targets[i][2],targets[i][3]])
        json_data['annotations'].append(val)
#         print(f"--{XcenYcenWH_to_XminYmin_WH([targets[i][0],targets[i][1],targets[i][2],targets[i][3]])}")
        with open('/home/host_data/PET_data/Neck/test/test.json', 'w', encoding='utf-8') as make_file:
            json.dump(json_data, make_file, indent="\t")
#     print(f"image shape:{image.shape}, label:{label_filename}, bbox:{targets[i][0:4]}, class_ids:{targets[i][4]}\n")

    
# with open('/home/host_data/PET_data/Body/test/test.json', 'r') as f:
#         json_data = json.load(f)
# print(json.dumps(json_data['annotations'], indent="\t"))